In my one of my previous [posts](https://dhruvdcoder.github.io/machinelearning/2018/03/17/linear_reg_using_gradient-mini-batch.html) we built a simple linear regression model using TensorFlow. Now, if you are working on a big project it becomes very important to structure your models in a consistent way so that they can be used with higherlevel TensorFlow API's like [tf.estimator.Estimator](https://www.tensorflow.org/programmers_guide/estimators). These highlevel APIs provide two major features:

1. They provide features like [summary](https://www.tensorflow.org/api_guides/python/summary) and [checkpointing](https://www.tensorflow.org/get_started/checkpoints) without you writing any extra code.

2. They keep the model and the runtime environment separated. What this means is that you can run training/evaluation/prediction on any environment (CPU, GPU, multiple GPUs) simply by changing a couple of lines in the configuration that you provide to the estimator, without making any change in your model code. More on this in the [TF Summit 2018](https://www.youtube.com/watch?v=4oNdaQk0Qv4).

Also, this kind of structure will help you experiment quickly with different models by getting rid of a lot of boilerplate code. So lets get started.


In this post I will be describing the following: 

1. Creating a standard model interface.  

2. Using this interface to structure a linear regression model.

3. Using the model with [tf.estimator.Estimator](https://www.tensorflow.org/programmers_guide/estimators).

## Creating a standard model interface

In [1]:
import abc

# create an abstract base class


class ModelBase(abc.ABC):

    def __init__(self,  *args, **kwargs):
        """ To store all the configuration parameters as instance attributes
            
        """
        
    @abc.abstractmethod
    def __call__(self, features, labels, mode, params):
        """Create the graph, send in the input and return the appropriate tensor
           depending on the mode

           Arguments:
               inputs:
           Returns:

           The graph has to be created only once. When the method is called the next time,
           it should simply execute the graph on the inputs.
        """
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            return self._call_train(features,labels)
        elif mode == tf.estimator.ModeKeys.PREDICT:
            return self._call_predict(features, labels)
        elif mode == tf.estimator.ModeKeys.EVAL:
            return self._call_eval(features,labels)
    
    
    #@abc.abstractmethod
    #def _create_variables(self, input_shape, *args, **kwargs):
    #    """Used to create variables for the model. Should be called only once."""
    #    pass
    
    @abc.abstractmethod
    def _call_train(self, inputs, targets):
        pass
    
    @abc.abstractmethod
    def _call_predict(self, inputs):
        pass
    
    @abc.abstractmethod
    def _call_eval(self,inputs, targets):
        pass
    

## Defining a Linear Regression model using the Model Interface

In [3]:
import tensorflow as tf


class LinearRegModel(ModelBase):
    def __init__(self):
        super().__init__()
        self._model_variable_scope = 'weights'
        self._kernel_name = 'kernel'
        self._bias_name = 'bias'
        self._apply_name = 'apply'
        self._kernel = None
        self._bias = None
        self._loss = lambda y_target, y_pred: tf.reduce_mean(
            tf.square(y_target - y_pred))
        self._optimizer = tf.train.AdamOptimizer
        self._learning_rate = 0.001
        self._metric = tf.metrics.root_mean_squared_error
        self._metric_name = 'RMS'

    def __call__(self, features, labels, mode, params):
        return super().__call__(features, labels, mode, params)

    def _call_predict(self, features, labels, is_internal_call=False):
        self._create_variables(features, labels)
        with tf.variable_scope(self._model_variable_scope, reuse=True) as scope:
            kernel = tf.get_variable(self._kernel_name, dtype=tf.float64)
            bias = tf.get_variable(self._bias_name, dtype=tf.float64)
            with tf.name_scope(self._apply_name):
                prediction = tf.add(tf.matmul(features, kernel), bias)
        if is_internal_call:
            return prediction
        else:
            mode = tf.estimator.ModeKeys.PREDICT
            return tf.estimator.EstimatorSpec(mode, predictions=prediction)

    def _call_eval(self, features, targets):
        y_pred = self._call_predict(features, targets, is_internal_call=True)
        loss = self._loss(targets, y_pred)
        metric = self._metric(targets, y_pred)
        mode = tf.estimator.ModeKeys.EVAL
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops={self._metric_name: metric})

    def _call_train(self, features, targets):
        y_pred = self._call_predict(features, targets, is_internal_call=True)
        loss = self._loss(targets, y_pred)
        training_op = self._optimizer(
            learning_rate=self._learning_rate).minimize(loss, global_step=tf.train.get_global_step())
        mode = tf.estimator.ModeKeys.TRAIN
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=training_op)

    def _infer_num_features(self, features):
        features_shape = features.get_shape()
        n_features = features_shape[1].value
        return n_features

    def _infer_num_labels(self, labels):
        label_shape = labels.get_shape()
        n_labels = 1
        if len(label_shape) > 1:
            n_labels = label_shape[1].value
        return n_labels

    def _create_variables(self, features, labels, *args, **kwargs):
        n_features = self._infer_num_features(features)
        n_labels = 1
        if labels is not None:
            n_labels = self._infer_num_labels(labels)
        with tf.variable_scope(self._model_variable_scope) as scope:
            self._kernel = tf.get_variable(name=self._kernel_name, initializer=tf.random_uniform(
                shape=[n_features, 1], dtype=tf.float64))
            self._bias = tf.get_variable(
                name=self._bias_name, initializer=tf.random_uniform([1], dtype=tf.float64))

## Using the Linear Regression model with `tf.estimator.Estimator`

In [4]:
# Fetching the data
from sklearn.datasets import fetch_california_housing
data=fetch_california_housing()

In [5]:
# Preprocessing the data
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_features=scaler.fit_transform(data.data)

In [6]:
train_test_ratio = 0.7
import numpy as np
((train_features, test_features), (train_y, test_y)) = (map(lambda x: np.split(
        x, [int(np.ceil(train_test_ratio*len(x)))]), (scaled_features, data.target)))

In [7]:
def train_input_fn(features, targets, batch_size):
    #create a Dataset object
    with tf.name_scope('train_input_pipline'):
        dataset = tf.data.Dataset.from_tensor_slices((features,targets))
        # repeat infinitely, shuffle and batch the data
        dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset

def eval_input_fn(features, targets, batch_size):
    with tf.name_scope('eval_input_pipeline'):
        dataset = tf.data.Dataset.from_tensor_slices((features, targets))
        dataset = dataset.batch(batch_size)
    return dataset

In [8]:
# create a model using LinearRegModel
model = LinearRegModel()
max_steps = 1000
batch_size = 500

In [9]:
estimator = tf.estimator.Estimator(model_fn=model, model_dir='temp/LinearReg')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_session_config': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_master': '', '_service': None, '_global_id_in_cluster': 0, '_task_id': 0, '_num_worker_replicas': 1, '_is_chief': True, '_tf_random_seed': None, '_save_checkpoints_secs': 600, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': 'temp/LinearReg', '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f503b878128>}


In [11]:
trained = estimator.train(lambda:train_input_fn(train_features,train_y,batch_size), max_steps=max_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into temp/LinearReg/model.ckpt.
INFO:tensorflow:step = 1, loss = 6.863418841271151
INFO:tensorflow:global_step/sec: 39.6945
INFO:tensorflow:step = 101, loss = 3.5302526225355915 (2.523 sec)
INFO:tensorflow:global_step/sec: 40.9672
INFO:tensorflow:step = 201, loss = 3.123366159133359 (2.441 sec)
INFO:tensorflow:global_step/sec: 41.4224
INFO:tensorflow:step = 301, loss = 4.016394770009598 (2.414 sec)
INFO:tensorflow:global_step/sec: 41.403
INFO:tensorflow:step = 401, loss = 2.2798705590292294 (2.415 sec)
INFO:tensorflow:global_step/sec: 41.1189
INFO:tensorflow:step = 501, loss = 4.009098033533354 (2.432 sec)
INFO:tensorflow:global_step/sec: 41.745
INFO:tensorflow:step = 601, loss = 2.7471322699039638 (2.396 sec)
IN

In [12]:
evaluated = estimator.evaluate(lambda:eval_input_fn(test_features,test_y,batch_size))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-31-11:59:57
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from temp/LinearReg/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-03-31-11:59:58
INFO:tensorflow:Saving dict for global step 1000: RMS = 1.6993942, global_step = 1000, loss = 2.927359
